## Multilayer Perceptron

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random

random.seed(1)
np.random.seed(1)
tf.random.set_seed(1)

2022-07-27 14:34:38.357539: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train, x_test = x_train.astype("float32"), x_test.astype("float32")

num_classes = 10
num_feature = 784

print(x_train.shape)
print(x_test.shape)
x_train, x_test = tf.reshape(x_train, [-1, num_feature]), tf.reshape(x_test, [-1, num_feature])
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28)
(10000, 28, 28)
(60000, 784)
(10000, 784)


2022-07-27 14:34:40.517382: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-07-27 14:34:40.563960: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:03:00.0 name: GeForce RTX 2080 computeCapability: 7.5
coreClock: 1.71GHz coreCount: 46 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.23GiB/s
2022-07-27 14:34:40.564613: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 1 with properties: 
pciBusID: 0000:04:00.0 name: GeForce RTX 2080 computeCapability: 7.5
coreClock: 1.71GHz coreCount: 46 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.23GiB/s
2022-07-27 14:34:40.564636: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-07-27 14:34:40.566046: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-07

In [3]:
x_train, x_test = x_train/255, x_test/255

In [4]:
class NeuralNet(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.fc1 = tf.keras.layers.Dense(128, activation=tf.nn.relu)
        self.fc2 = tf.keras.layers.Dense(256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(10)
        
    def call(self, x, is_training=False):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.out(x)
        
        if not is_training:
            x = tf.nn.softmax(x)
        
        return x

In [5]:
neural_net = NeuralNet()

In [6]:
def cross_entropy_loss(x, y):
    y = tf.cast(y, tf.int64)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=x)
    return tf.reduce_mean(loss)

In [7]:
def accuracy(y_pred, y_true):
    correct_prediction = tf.equal(tf.argmax(y_pred, axis=1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [8]:
lr = 0.005
optimizer = tf.optimizers.SGD(learning_rate=lr)

def run_optimization(x, y):
    with tf.GradientTape() as g:
        pred = neural_net(x, is_training=False)
        loss = cross_entropy_loss(pred, y)
    
    parameters = neural_net.trainable_variables
    # gradients = g.gradient(loss, [W, b])
    gradients = g.gradient(loss, parameters)
    optimizer.apply_gradients(zip(gradients, parameters))


In [9]:
from tqdm import tqdm

In [10]:
batch_size = 200
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.shuffle(60000).batch(batch_size).prefetch(1)

epoch = 40
display_epoch = 4

for epo in tqdm(range(1, epoch+1)):
    for step, (batch_x, batch_y) in enumerate(train_data, 1):
        run_optimization(batch_x, batch_y)
    
    if epo % display_epoch == 0:
        pred = neural_net(batch_x)
        loss = cross_entropy_loss(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("epoch :", epo)
        print("loss :", loss.numpy())
        print("accuracy :", acc.numpy())

 10%|████▍                                       | 4/40 [00:09<01:26,  2.41s/it]

epoch : 4
loss : 2.2527626
accuracy : 0.27


 20%|████████▊                                   | 8/40 [00:19<01:18,  2.45s/it]

epoch : 8
loss : 2.1001873
accuracy : 0.415


 30%|████████████▉                              | 12/40 [00:29<01:08,  2.46s/it]

...

epoch : 32
loss : 1.6705561
accuracy : 0.825


 90%|██████████████████████████████████████▋    | 36/40 [01:28<00:09,  2.48s/it]

epoch : 36
loss : 1.636029
accuracy : 0.855


100%|███████████████████████████████████████████| 40/40 [01:38<00:00,  2.47s/it]

epoch : 40
loss : 1.6501181
accuracy : 0.82


In [11]:
pred = neural_net(x_test)
print("Test Accuracy : ", accuracy(pred, y_test).numpy())

Test Accuracy :  0.82


## Convolution Neural Network

In [14]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [18]:
def convolution_layer_test(filters, kernel_size, s=1, p="valid", input_size=[1, 64, 64, 3]):
    inputs = tf.random.normal(input_size)
    print("input size : ", inputs.shape)
    conv = tf.keras.layers.Conv2D(filters, kernel_size, strides=s, padding=p)
    outputs = conv(inputs)
    print("output size : ", outputs.shape)

In [19]:
convolution_layer_test(10, 3)  ## (W-F+2P)/S + 1  --> (64-3+2*0)/1 + 1 = 62

input size :  (1, 64, 64, 3)
output size :  (1, 62, 62, 10)


In [22]:
convolution_layer_test(5, 11, s=6, p="valid", input_size=[1, 128, 128, 3])

input size :  (1, 128, 128, 3)
output size :  (1, 20, 20, 5)


In [25]:
def pooling_layer_test(pool_size, s=1, p="valid", input_size=[1, 64, 64, 3]):
    inputs = tf.random.normal(input_size)
    print("input size : ", inputs.shape)
    pool = tf.keras.layers.MaxPooling2D(pool_size, strides=s, padding=p)
    outputs = pool(inputs)
    print("output size : ", outputs.shape)

In [26]:
pooling_layer_test(2, 2)

input size :  (1, 64, 64, 3)
output size :  (1, 32, 32, 3)
